<a href="https://colab.research.google.com/github/lookinsight/ml/blob/main/20221116_ML_K%ED%8F%89%EA%B7%A0%EA%B5%B0%EC%A7%91%ED%99%94_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 

In [ ]:
# https://raw.githubusercontent.com/bigdata-young/bigdata_16th/main/data/example_cluster.csv
file_url = "https://raw.githubusercontent.com/bigdata-young/bigdata_16th/main/data/example_cluster.csv"
df = pd.read_csv(file_url) 

In [ ]:
df.head()

In [ ]:
sns.scatterplot(x = 'var_1', y='var_2', data = df) 

In [ ]:
# K 평균 군집화
from sklearn.cluster import KMeans 

In [ ]:
kmeans_model = KMeans(n_clusters = 3, random_state = 100) 

In [ ]:
kmeans_model.fit(df) 

In [ ]:
pred = kmeans_model.predict(df) 
pred

In [ ]:
df['label'] = pred

In [ ]:
sns.scatterplot(x='var_1', y='var_2', data = df,
                hue='label', palette = 'rainbow') 

In [ ]:
# 이녀셔(inertia) 
# 각 그룹에서 중심과 각 그룹에 해당하는 데이터 간 거리에 대한 합 
kmeans_model.inertia_

In [ ]:
kmeans_model2 = KMeans(n_clusters = 500, random_state = 100) 
kmeans_model2.fit(df) 
kmeans_model2.inertia_

In [ ]:
df['label2'] = kmeans_model2.predict(df) 

In [ ]:
df['label2']

In [ ]:
sns.scatterplot(x='var_1', y='var_2', data = df,
                hue='label2', palette = 'rainbow') 

In [ ]:
# 엘보우 기법
df = pd.read_csv(file_url) 
distance = []
for k in range(2, 10): # k: 2~9
    k_model = KMeans(n_clusters=k)
    k_model.fit(df)
    distance.append(k_model.inertia_)

In [ ]:
distance

In [ ]:
sns.lineplot(x=range(2,10), y=distance)

In [ ]:
file_url = "https://raw.githubusercontent.com/bigdata-young/bigdata_16th/main/data/customer.csv"
df = pd.read_csv(file_url)

In [ ]:
df.head()

In [ ]:
df.describe(include = 'all') 

In [ ]:
df.cc_num.nunique()

In [ ]:
# cust_dummy = pd.get_dummies(df, columns = ['category'], drop_first = True)
cust_dummy = pd.get_dummies(df, columns = ['category'])  
cust_dummy.info()

In [ ]:
cust_dummy.columns[2:]

In [ ]:
cat_list = cust_dummy.columns[2:]

In [ ]:
for i in cat_list:  # 열들의 이름
    # cust_dummy: 0, 1 * 금액 -> 그 카테고리에 속한 금액 
    cust_dummy[i] = cust_dummy[i] * cust_dummy['amt'] 

In [ ]:
cust_dummy 

In [ ]:
customer_agg = cust_dummy.groupby('cc_num').sum()
customer_agg.head()

In [ ]:
customer_agg.info()

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() 
scaled_df = pd.DataFrame(scaler.fit_transform(customer_agg),
                         columns = customer_agg.columns,
                         index = customer_agg.index)
scaled_df

In [ ]:
distance = []
for k in range(2, 10): 
    k_model = KMeans(n_clusters = k)
    k_model.fit(scaled_df) 
    labels = k_model.predict(scaled_df) 
    distance.append(k_model.inertia_) 

sns.lineplot(x = range(2, 10), y = distance) 

In [ ]:
# 실루엣 계수 
# - 이너셔: 그룹의 중심으로부터 외부 요소 간의 거리...
# - 클러스터 냅의 평균 거리 + 최근접한 다른 클러스터 데이터의 평균 거리 
from sklearn.metrics import silhouette_score

In [ ]:
silhouette = []
for k in range(2, 10):
    k_model = KMeans(n_clusters=k)
    k_model.fit(scaled_df)
    labels = k_model.predict(scaled_df)
    silhouette.append(silhouette_score(scaled_df, labels))

In [ ]:
silhouette

In [ ]:
# 실루엣 계수는 더 높은 값일 수록 더 좋은 분류
sns.lineplot(x=range(2, 10), y=silhouette)

In [ ]:
k_model = KMeans(n_clusters = 4) 
k_model.fit(scaled_df) 
labels = k_model.predict(scaled_df) 

In [ ]:
scaled_df['label'] = labels

In [ ]:
scaled_df.head()

In [ ]:
scaled_df_mean = scaled_df.groupby('label').mean()
scaled_df_mean

In [ ]:
scaled_df_count = scaled_df.groupby('label').count().amt
scaled_df_count

In [ ]:
scaled_df_count = scaled_df_count.rename('count')

In [ ]:
scale_df_all = scaled_df_mean.join(scaled_df_count)
scale_df_all